In [1]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch.utils.data.sampler import WeightedRandomSampler
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# 数据预处理
train_transform = transforms.Compose([
    transforms.Resize((90, 90)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((90, 90)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
#计算每个类别的图片数量
train_path = 'D:\Csci323\Emotion-domestic\Emotion-domestic\Train'
train_dataset = ImageFolder(root=train_path, transform=train_transform)
# 计算每个类别的权重
class_sample_counts = [3422, 3285, 1602, 24034, 10908, 2875, 3475]
weights = 1. / np.array(class_sample_counts)
samples_weights = np.array([weights[label] for _, label in train_dataset.samples])
# 创建采样器
sampler = WeightedRandomSampler(weights=samples_weights, num_samples=len(samples_weights) * 2, replacement=True)
# 使用采样器创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=4)

test_dataset = ImageFolder(root='D:\Csci323\Emotion-domestic\Emotion-domestic\Test', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [5]:
# 加载预训练的ResNet50模型
model = models.resnet50(pretrained=True)

# 修改最后的全连接层以适应FER-2013的7个类别
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)

#将模型加载到GPU
model = model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

d:\Users\0\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Users\0\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# 训练模型
for epoch in range(20):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs,labels = inputs.to(device),labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1} loss: {running_loss / 3100:.3f}')
    # 测试模型
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images,labels = images.to(device),labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the test images: {100 * correct / total:.2f}%')
print('Finished Training')

Epoch 1 loss: 0.559
Accuracy of the network on the test images: 20.50%
Epoch 2 loss: 0.267
Accuracy of the network on the test images: 22.98%
Epoch 3 loss: 0.180
Accuracy of the network on the test images: 27.30%
Epoch 4 loss: 0.137
Accuracy of the network on the test images: 29.42%
Epoch 5 loss: 0.107
Accuracy of the network on the test images: 28.60%
Epoch 6 loss: 0.086
Accuracy of the network on the test images: 31.78%
Epoch 7 loss: 0.073
Accuracy of the network on the test images: 30.50%
Epoch 8 loss: 0.064
Accuracy of the network on the test images: 32.88%
Epoch 9 loss: 0.056
Accuracy of the network on the test images: 35.18%
Epoch 10 loss: 0.044
Accuracy of the network on the test images: 33.40%
Epoch 11 loss: 0.048
Accuracy of the network on the test images: 34.26%
Epoch 12 loss: 0.044
Accuracy of the network on the test images: 37.34%
Epoch 13 loss: 0.034
Accuracy of the network on the test images: 37.42%
Epoch 14 loss: 0.042
Accuracy of the network on the test images: 37.94%
E

In [7]:
torch.save(model, 'D:\Csci323\ResNET_50.pth')